Import Libraries

In [ ]:
import os, glob, shutil 

Get all images set

In [24]:
images_names = os.listdir('/mnt/16F2E723F2E70639/ITI/Gradution Project/Scouting/sub-scout')
images_set = {img_name[:img_name.rfind('-')] for img_name in images_names}

print(sorted(images_set))

['1', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', '2', '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '3', '3-1', '3-2', '3-3', '3-4', '3-5', '3-6', '4', '4-1', '4-2', '4-3', '4-4', '4-5', '4-6', '5', '6']


Get images set that have been annotated


In [16]:
images_names = os.listdir('/mnt/16F2E723F2E70639/ITI/Gradution Project/Scout4Stars/dataset/images')
annotated_images_set = {img_name[:img_name.rfind('-')] for img_name in images_names}

Get images set that needs to be annotated to be marked as reference later


In [22]:
print(sorted(images_set - annotated_images_set))

['2', '2-2', '2-5', '2-6', '4', '4-1', '4-2', '4-3', '4-4', '4-5', '4-6', '5', '6']


So we need to annotate image for every image in from the following set:

`['2', '2-2', '2-5', '2-6', '4', '4-1', '4-2', '4-3', '4-4', '4-5', '4-6', '5', '6']`

to be marked as reference.

We excluded `['1-3', '1-5', '2-3', '3-5', '2-5', '4-3', '4-5', '5']` from the set as it has no static objects over all the set.

In [282]:
references = [
                '1-00351.txt', '1-1-00039.txt', '1-2-00044.txt', '1-4-00006.txt', 
                '1-6-00005.txt', '2-00153.txt', '2-1-00034.txt', '2-2-00059.txt',
                '2-4-00018.txt', '2-6-00002.txt', '3-00001.txt', '3-1-00214.txt',
                '3-2-00022.txt', '3-3-00003.txt', '3-4-00630.txt', '3-6-00002.txt',
                '4-00073.txt', '4-1-00110.txt', '4-2-00203.txt', '4-4-00060.txt',
                '4-6-00006.txt', '6-00106.txt'
            ]

Convert references to dictionary with respect to sets as keys

In [283]:
references = {reference[:reference.rfind('-')]:reference for reference in references}
references

{'1': '1-00351.txt',
 '1-1': '1-1-00039.txt',
 '1-2': '1-2-00044.txt',
 '1-4': '1-4-00006.txt',
 '1-6': '1-6-00005.txt',
 '2': '2-00153.txt',
 '2-1': '2-1-00034.txt',
 '2-2': '2-2-00059.txt',
 '2-4': '2-4-00018.txt',
 '2-6': '2-6-00002.txt',
 '3': '3-00001.txt',
 '3-1': '3-1-00214.txt',
 '3-2': '3-2-00022.txt',
 '3-3': '3-3-00003.txt',
 '3-4': '3-4-00630.txt',
 '3-6': '3-6-00002.txt',
 '4': '4-00073.txt',
 '4-1': '4-1-00110.txt',
 '4-2': '4-2-00203.txt',
 '4-4': '4-4-00060.txt',
 '4-6': '4-6-00006.txt',
 '6': '6-00106.txt'}

Get annotations of the chosen reference images

In [224]:
references_path = os.path.join(os.path.join(os.pardir, 'dataset'), 'references')
references_images_path = os.path.join(references_path, 'images')
references_annotations_path = os.path.join(references_path, 'annotations')

source_path = os.path.join(os.pardir, 'dataset')
source_annotations_path = os.path.join(source_path, 'annotations')

source_annotations = os.listdir(source_annotations_path)
references_images = os.listdir(os.path.join(references_path, 'images'))

os.makedirs(references_annotations_path, exist_ok=True)

for img in references_images:
    annotation = img.replace('jpg', 'txt')
    if annotation in source_annotations:
        src = os.path.join(source_annotations_path, annotation)
        shutil.copy(src, references_annotations_path)

Remove non static objects (players) from references annotations files

In [ ]:
# references_annotations = glob.glob(os.path.join(os.path.join(references_path, 'annotations'), '*txt'))

# for annotation in references_annotations:
#     with open(annotation, 'r+') as f:
#         lines = f.readlines()
#         f.seek(0)
#         for line in lines:
#             if not line.startswith('0'):
#                 f.write(line)
#         f.truncate()

Create semi-annotated files

In [290]:
dataset_path = '/mnt/16F2E723F2E70639/ITI/Gradution Project/Scouting/sub-scout/'

source_annotations_path = os.path.join(source_path, 'annotations')

dest_path = os.path.join(source_path, 'semi-annotations')
os.makedirs(dest_path, exist_ok=True)

for img in os.listdir(dataset_path):
    annotation = img.replace('jpg', 'txt')
    reference_annotation = references.get(annotation[:annotation.rfind('-')])
    if reference_annotation:
        src = os.path.join(source_annotations_path, reference_annotation)
        # copy reference annotation file to destination
        shutil.copy(src, dest_path)

        # rename the file with the annotation name
        old_name = os.path.join(dest_path, reference_annotation)
        new_name = os.path.join(dest_path, annotation)
        os.rename(old_name, new_name)

        # remove non static objects (players) from semi-annotations files
        with open(new_name, 'r+') as f:
            lines = f.readlines()
            f.seek(0)
            for line in lines:  
                if not line.startswith('0'):
                    f.write(line)
            f.truncate()